# Dog and Cat Dataset을 가지고 영상처리 실습

Download Cat and Dog Dataset: https://www.kaggle.com/c/dogs-vs-cats/data 에서 다운로드 받기


In [1]:
import tensorflow as tf
import numpy as np
import matplotlib
from matplotlib.pyplot import *
import matplotlib.pyplot as plt
import glob
import re
import os

# use matplotlib as inline
%matplotlib inline 
from functools import partial

In [37]:
def get_cat_and_dog_files(data):

    cat_files = []
    dog_files = []

    for string in data:

        result1 = re.search("cat", string)
        result2 = re.search("dog", string)
        
        if result1 is not None:
            cat_files.append(string)
        if result2 is not None:
            dog_files.append(string)   

    return cat_files, dog_files      

In [38]:
import os        # os에 관계된 라이브러리들을 import한다.  
import glob
dirname = 'data/train'   # 디렉토리명
dataset_path = "data/train/*.jpg"
#dataset_path = "*.jpg"
data = glob.glob(dataset_path)

# dirname이라는 디렉토리 안에 있는 모든 파일을 filenames에 저장한다. 
filenames = [os.path.join(dirname, fname)   # 밑에서 얻은 파일들을 [](배열)안에 추가 
             for fname in os.listdir(dirname)] 
#print(filenames)
#data = glob.glob(dirname)
print("{} images found:".format(len(data)))
#print(filenames)

25000 images found:


In [ ]:
# test image import
dirname_test = 'data/test'
dataset_path2 = "data/test/*.jpg"
data_test = glob.glob(dataset_path2)

# dirname이라는 디렉토리 안에 있는 모든 파일을 filenames에 저장한다. 
filenames_test = [os.path.join(dirname_test, fname)   # 밑에서 얻은 파일들을 [](배열)안에 추가 
             for fname in os.listdir(dirname_test)] 
#print(filenames)
#data = glob.glob(dirname)
print("{} images found:".format(len(data_test)))

In [39]:
#filenames섞기
import random

random.shuffle(filenames)
#print(filenames)
random.shuffle(filenames)
random.shuffle(filenames)
#print(filenames)

In [40]:
# label만들기
import re

def get_labels(data):

    labels = []

    for string in data:

        result1 = re.search("cat", string)
        result2 = re.search("dog", string)
        
        if result1 is not None:
            labels.append([1.0, 0.0])
        if result2 is not None:
            labels.append([0.0, 1.0])

    return labels

labels = get_labels(filenames) # label 만들기
labels_test = get_labels(filenames_test) # test label 만들기

In [42]:
from scipy import misc # feel free to use another image loader

batch_size = 50
total = 24900
total_batch = int(total/batch_size)

def create_batches(list_of_images,labels,batch_size):
    images = []
    img_size = (224, 224, 3)
    for img in list_of_images:
        images.append(misc.imresize(misc.imread(img),img_size))
    
    images = np.asarray(images)
    images = images.astype(np.float32)
    images[:,:,:,0] /= 255.    
    images[:,:,:,1] /= 255.  
    images[:,:,:,2] /= 255.    
    images[:,:,:,0] -= 0.5    
    images[:,:,:,1] -= 0.5
    images[:,:,:,2] -= 0.5
    images[:,:,:,0] *= 2.0    
    images[:,:,:,1] *= 2.0
    images[:,:,:,2] *= 2.0
    
    while (True):
        for i in range(0,total,batch_size):
            yield(images[i:i+batch_size],labels[i:i+batch_size])
            
def create_test_images(list_of_images):
    images = []
    img_size = (224, 224, 3)
    for img in list_of_images:
        images.append(misc.imresize(misc.imread(img),img_size))
    
    images = np.asarray(images)
    images = images.astype(np.float32)
    images[:,:,:,0] /= 255.    
    images[:,:,:,1] /= 255.  
    images[:,:,:,2] /= 255.    
    images[:,:,:,0] -= 0.5    
    images[:,:,:,1] -= 0.5
    images[:,:,:,2] -= 0.5
    images[:,:,:,0] *= 2.0    
    images[:,:,:,1] *= 2.0
    images[:,:,:,2] *= 2.0
    
    return images

test_images = create_test_images(filenames_test)

In [ ]:
# Loss 함수
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_))

#cross_entropy = -tf.reduce_sum(y_*tf.log(y))
#cross_entropy = tf.reduce_mean((y-y_)**2)

#checkpoint를 저장하는 위치
# ckpt = tf.train.get_checkpoint_state('./cnn')
# global_variables 함수를 통해 앞서 정의하였던 변수들을 저장하거나 불러올 변수들로 설정함.
saver = tf.train.Saver(tf.global_variables())
global_step = tf.Variable(0, trainable=False, name='global_step')


number_of_epochs = 10000
#batch normalization을 제대로 하기 위해 graph에 mean average를 update하라고 말한다.
# http://ruishu.io/2016/12/27/batchnorm/ 를 참조하기 
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

with tf.control_dependencies(update_ops):
     # Ensures that we execute the update_ops before performing the train_step
     train_step = tf.train.AdamOptimizer(0.001).minimize(cost, global_step=global_step)
    
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(number_of_epochs):

#checkpoint가 있으면 restore하고 아니면 변수들을 초기화한다. 
#    if ckpt:
#        saver.restore(sess, ckpt.model_checkpoint_path)
#    else:
#        sess.run(tf.global_variables_initializer())
    random.shuffle(filenames)
    labels = get_labels(filenames) # label 만들기
    batch_generator = create_batches(filenames,labels,batch_size)
    
    total_cost=0
    start_time = time.time()
    for i in range(total_batch):
        images, labels2 = next(batch_generator)
        _, cost_val = sess.run([train_step,cost],
                feed_dict={x : images, 
                           y_: labels2, 
                           phase: True})    
        total_cost += cost_val
    
    print('Epoch:', '%04d' % (epoch + 1),
          'Avg. cost =', '{:.3f}'.format(total_cost))
    print("--- %s seconds ---" % (time.time() - start_time))
  
    acc = sess.run(accuracy, feed_dict={x: test_images, y_: labels_test, phase: False})
    print('Accuracy:', acc) # training batch에 대한 정확도 출력....(batch사이즈가 작아서 정확하지 않아도 됨.)
    
#변수를 checkpoint파일에 저장.    
#    saver.save(sess, './cnn/dnn.ckpt', global_step=global_step)

print('End of optimization!')

# Noisy Image 만들기

In [ ]:
# noise영상을 사용하려면 이것을 하라.
img_noise = np.random.uniform(size=(224,224,3)) + 100.0
img_4d = img_noise[np.newaxis]
# Rescale to 0-1 range
img_4d = img_4d / np.max(img_4d)

In [ ]:
import PIL.Image
img0 = PIL.Image.open('pilatus224.jpg')
img0 = np.float32(img0)
img_4d = img0[np.newaxis]
img_4d = img_4d / 255.0
img_4d_0 = img_4d/255
img_4d[:,:,:,0] /= 255.    
img_4d[:,:,:,1] /= 255.  
img_4d[:,:,:,2] /= 255.    
img_4d[:,:,:,0] -= 0.5    
img_4d[:,:,:,1] -= 0.5
img_4d[:,:,:,2] -= 0.5
img_4d[:,:,:,0] *= 2.0    
img_4d[:,:,:,1] *= 2.0
img_4d[:,:,:,2] *= 2.0
#img_4d = img_4d_0 / np.max(img_4d_0)
#img_4d_0 = img_4d_0
plt.imshow(visstd(img_4d[0]))
print(img_4d.shape)

In [ ]:
def visstd(a, s=0.1):
    '''Normalize the image range for visualization'''
    return (a-a.mean())/max(a.std(), 1e-4)*s + 0.5

In [ ]:
def plot_image_grid(images_files):
    # figure size
    fig = plt.figure(figsize=(8, 8))
    
    # load images
    images = [tf_keras.preprocessing.image.load_img(img) for img in images_files]
    
    # plot image grid
    for x in range(4):
        for y in range(4):
            ax = fig.add_subplot(4, 4, 4*y+x+1)
            plt.imshow(images[4*y+x])
            plt.xticks(np.array([]))
            plt.yticks(np.array([]))
    plt.show()

In [ ]:
plot_image_grid(cat_files[:16])